In [1]:
import os
import time

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd

from credentials import set_credentials

In [2]:
pd.options.display.max_columns = None

# Spotify Recommendation System Walkthrough

## Spotify Access
How to access Spotify through their developer API

In [3]:
set_credentials()

In [4]:
client_credentials_manager = SpotifyClientCredentials(client_id=os.getenv('CLIENT_ID'), client_secret=os.getenv('CLIENT_SECRET'))
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### Extract Tracks from a Playlist

Spotify makes use of a URI (Universal resource locator) in order to identify playlsits, artists, tracks, ect. For example from the link: `https://open.spotify.com/playlist/2QJwkjEtDXDbtecQ5ypVm3?si=c437b89583c74615` to one of my favourite playlists, only the `2QJwkjEtDXDbtecQ5ypVm3` is the URI.

In [5]:
playlist_link = "https://open.spotify.com/playlist/2QJwkjEtDXDbtecQ5ypVm3?si=c437b89583c74615"  # Motivate
# playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF?si=1333723a6eff4b7f"  # Tutorial

In [6]:
playlist_uri = playlist_link.split('/')[-1].split('?')[0]
print(playlist_uri)

2QJwkjEtDXDbtecQ5ypVm3


In [7]:
mot_playlist = sp.playlist_tracks(playlist_uri, limit=100, offset=0)

In [8]:
print(mot_playlist)

{'href': 'https://api.spotify.com/v1/playlists/2QJwkjEtDXDbtecQ5ypVm3/tracks?offset=0&limit=100&additional_types=track', 'items': [{'added_at': '2020-11-14T05:40:14Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/fn662s8jr8skpfl5kn81itq7c'}, 'href': 'https://api.spotify.com/v1/users/fn662s8jr8skpfl5kn81itq7c', 'id': 'fn662s8jr8skpfl5kn81itq7c', 'type': 'user', 'uri': 'spotify:user:fn662s8jr8skpfl5kn81itq7c'}, 'is_local': False, 'primary_color': None, 'track': {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3NPpFNZtSTHheNBaWC82rB'}, 'href': 'https://api.spotify.com/v1/artists/3NPpFNZtSTHheNBaWC82rB', 'id': '3NPpFNZtSTHheNBaWC82rB', 'name': 'X Ambassadors', 'type': 'artist', 'uri': 'spotify:artist:3NPpFNZtSTHheNBaWC82rB'}], 'available_markets': ['AD', 'AE', 'AG', 'AR', 'AT', 'AU', 'BB', 'BE', 'BG', 'BH', 'BO', 'BR', 'BS', 'BZ', 'CA', 'CH', 'CL', 'CO', 'CR', 'CW', 'CY', 'CZ', 'DE', 'DK', 'DM', 'DO', '

Display the total number of songs in the playlist

In [9]:
print(mot_playlist['total'])

301


Display author of playlist (spotify link)

In [10]:
items = mot_playlist['items']

In [11]:
print(items)

[{'added_at': '2020-11-14T05:40:14Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/fn662s8jr8skpfl5kn81itq7c'}, 'href': 'https://api.spotify.com/v1/users/fn662s8jr8skpfl5kn81itq7c', 'id': 'fn662s8jr8skpfl5kn81itq7c', 'type': 'user', 'uri': 'spotify:user:fn662s8jr8skpfl5kn81itq7c'}, 'is_local': False, 'primary_color': None, 'track': {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3NPpFNZtSTHheNBaWC82rB'}, 'href': 'https://api.spotify.com/v1/artists/3NPpFNZtSTHheNBaWC82rB', 'id': '3NPpFNZtSTHheNBaWC82rB', 'name': 'X Ambassadors', 'type': 'artist', 'uri': 'spotify:artist:3NPpFNZtSTHheNBaWC82rB'}], 'available_markets': ['AD', 'AE', 'AG', 'AR', 'AT', 'AU', 'BB', 'BE', 'BG', 'BH', 'BO', 'BR', 'BS', 'BZ', 'CA', 'CH', 'CL', 'CO', 'CR', 'CW', 'CY', 'CZ', 'DE', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GD', 'GR', 'GT', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS',

In [12]:
len(items)

100

### Extract Track Information (Depreciated)
The next cells (until the next heading) are depreciated as they use a step by step process rather than a batch process. A such it does not gather all of the data, but serves as a starting point of track exploration

In [13]:
track_info = {
    'uris': [],
    'names': [],
    'artist_names': [],
    'artists_uri': [], 
    'artist_pop': [],
    'artist_genres': [],
    'albums': [],
    'track_pop': []
}

In [14]:
for item in items:
    track_uri = item['track']['uri']
    track_info['uris'].append(track_uri)  # Retrieve track uri
    track_info['names'].append(item['track']['name'])  # Retrieve track name

    track_info['artists_uri'].append(item['track']['artists'][0]['uri'].split(':')[-1])  # Find artist uri

    track_info['artist_names'].append(item['track']['artists'][0]['name'])  # Access artist name


    track_info['albums'].append(item['track']['album']['name'])  # Access album names
    track_info['track_pop'].append(item['track']['popularity'])  # Access track popularity

The below code is used to extract the Artist details as well as a acoustic analysis of the songs. 
It is written in such a way that it makes an API call for each song instead of using a batch process. Due to this the code is removed to reduce the amount of API calls, and modified later in the code to perform batch requests. 

```python
# Artist Features
track_info['artist_pop'].append(artist_info['popularity'])  # Access 
track_info['artist_genres'].append(artist_info['genres'])  # Access artist genres

# Audio Features
track_features = sp.audio_features(track_uri)[0]  # Access track features
track_info['danceability'].append(track_features['danceability'])
track_info['energy'].append(track_features['energy'])
track_info['keys'].append(track_features['key'])
track_info['loudness'].append(track_features['loudness'])
track_info['modes'].append(track_features['mode'])
track_info['speechiness'].append(track_features['speechiness'])
track_info['acousticness'].append(track_features['acousticness'])
track_info['instrumentalness'].append(track_features['instrumentalness'])
track_info['liveness'].append(track_features['liveness'])
track_info['valences'].append(track_features['valence'])
track_info['tempos'].append(track_features['tempo'])
track_info['types'].append(track_features['type'])
track_info['ids'].append(track_features['id'])
track_info['track_hrefs'].append(track_features['track_href'])
track_info['analysis_urls'].append(track_features['analysis_url'])
track_info['durations_ms'].append(track_features['duration_ms'])
track_info['time_signatures'].append(track_features['time_signature'])
```

In [15]:
artist_info = sp.artists(track_info['artists_uri'][0:50])

In [16]:
artist_info['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/3NPpFNZtSTHheNBaWC82rB'},
  'followers': {'href': None, 'total': 2944194},
  'genres': ['modern alternative rock', 'modern rock', 'stomp pop'],
  'href': 'https://api.spotify.com/v1/artists/3NPpFNZtSTHheNBaWC82rB',
  'id': '3NPpFNZtSTHheNBaWC82rB',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab6761610000e5eb85b6cbf01747c44e145c5a20',
    'width': 640},
   {'height': 320,
    'url': 'https://i.scdn.co/image/ab6761610000517485b6cbf01747c44e145c5a20',
    'width': 320},
   {'height': 160,
    'url': 'https://i.scdn.co/image/ab6761610000f17885b6cbf01747c44e145c5a20',
    'width': 160}],
  'name': 'X Ambassadors',
  'popularity': 70,
  'type': 'artist',
  'uri': 'spotify:artist:3NPpFNZtSTHheNBaWC82rB'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/3NPpFNZtSTHheNBaWC82rB'},
  'followers': {'href': None, 'total': 2944194},
  'genres': ['modern alternative rock', 'modern rock', 'stomp pop'],

In [17]:
len(artist_info['artists'])

50

In [18]:
audio_features = sp.audio_features(track_info['uris'])

In [19]:
audio_features

[{'danceability': 0.732,
  'energy': 0.594,
  'key': 11,
  'loudness': -5.911,
  'mode': 0,
  'speechiness': 0.0693,
  'acousticness': 0.00805,
  'instrumentalness': 0.0301,
  'liveness': 0.0802,
  'valence': 0.676,
  'tempo': 107.996,
  'type': 'audio_features',
  'id': '3V9cM3nCH2G66afoDi0snu',
  'uri': 'spotify:track:3V9cM3nCH2G66afoDi0snu',
  'track_href': 'https://api.spotify.com/v1/tracks/3V9cM3nCH2G66afoDi0snu',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3V9cM3nCH2G66afoDi0snu',
  'duration_ms': 164587,
  'time_signature': 4},
 {'danceability': 0.526,
  'energy': 0.862,
  'key': 2,
  'loudness': -6.003,
  'mode': 1,
  'speechiness': 0.0905,
  'acousticness': 0.0144,
  'instrumentalness': 0.0597,
  'liveness': 0.229,
  'valence': 0.528,
  'tempo': 90.052,
  'type': 'audio_features',
  'id': '0fYVliAYKHuPmECRs1pbRf',
  'uri': 'spotify:track:0fYVliAYKHuPmECRs1pbRf',
  'track_href': 'https://api.spotify.com/v1/tracks/0fYVliAYKHuPmECRs1pbRf',
  'analysis_url': 'http

### DataFrame the Information

## Automate Playlist Track Data Extraction with Offsetting

In [20]:
def construct_storage():
    store_outline = {
        'uris': [],
        'names': [],
        'artist_names': [],
        'artist_uris': [],
        'artist_pop': [],
        'artist_genres': [],
        'albums': [],
        'track_pop': [],
        'danceability': [],
        'energy': [],
        'keys': [],
        'loudness': [],
        'modes': [],
        'speechiness': [],
        'acousticness': [],
        'instrumentalness': [],
        'liveness': [],
        'valences': [],
        'tempos': [],
        'types': [],
        'ids': [],
        'track_hrefs': [],
        'analysis_urls': [],
        'durations_ms': [],
        'time_signatures': [],
        'playlist_name': []
    }
    return store_outline

def process_items(store, items):
    for item in items:
        track_uri = item['track']['uri'].split(':')[-1]
        store['uris'].append(track_uri)  # Retrieve track uri
        store['names'].append(item['track']['name'])  # Retrieve track name

        store['artist_uris'].append(item['track']['artists'][0]['uri'].split(':')[-1])  # Find artist uri

        store['artist_names'].append(item['track']['artists'][0]['name'])  # Access artist name

        store['albums'].append(item['track']['album']['name'])  # Access album names
        store['track_pop'].append(item['track']['popularity'])  # Access track popularity
    return store


def retrieve_batch_info(playlist, store):
    items = playlist['items']
    store = process_items(store, items)
    return store


def extract_artist_info(store, sp):
    limit = 50
    offset = 0
    while offset < len(store['artist_uris']):
        if offset + limit > len(store['artist_uris']):
            artists_info = sp.artists(store['artist_uris'][offset: len(store['artist_uris'])])
        else:
            artists_info = sp.artists(store['artist_uris'][offset: offset + limit])

        for artist in artists_info['artists']:
            store['artist_pop'].append(artist['popularity'])  # Access artist popularity
            store['artist_genres'].append(artist['genres'])  # Access artist genres

        offset = offset + limit


def extract_audio_features(store, sp):
    limit = 100
    offset = 0
    while offset < len(store['uris']):
        if offset + limit > len(store['uris']):
            track_info = sp.audio_features(store['uris'][offset: len(store['uris'])])
        else:
            track_info = sp.audio_features(store['uris'][offset: offset + limit])

        for track in track_info:
            store['danceability'].append(track['danceability'])
            store['energy'].append(track['energy'])
            store['keys'].append(track['key'])
            store['loudness'].append(track['loudness'])
            store['modes'].append(track['mode'])
            store['speechiness'].append(track['speechiness'])
            store['acousticness'].append(track['acousticness'])
            store['instrumentalness'].append(track['instrumentalness'])
            store['liveness'].append(track['liveness'])
            store['valences'].append(track['valence'])
            store['tempos'].append(track['tempo'])
            store['types'].append(track['type'])
            store['ids'].append(track['id'])
            store['track_hrefs'].append(track['track_href'])
            store['analysis_urls'].append(track['analysis_url'])
            store['durations_ms'].append(track['duration_ms'])
            store['time_signatures'].append(track['time_signature'])

        offset = offset + limit

In [21]:
def extract_tracks(sp, playlist_uri, store):
    offset = 0
    limit = 100
    playlist = sp.playlist_tracks(playlist_uri, limit=2, offset=offset)  # Retrieve the initial batch of songs
    total_songs = playlist['total']  # Extract the total number of songs
    print(f"Total songs: {total_songs}")

    while offset < total_songs:
        time.sleep(2)
        playlist = sp.playlist_tracks(playlist_uri, limit=100, offset=offset)  # Retrieve batch of songs in playlist
        store = retrieve_batch_info(playlist, store)  # Retrieve batch information
        print(f"Current offset: {offset}")
        offset = offset + limit  # Update offset

    extract_artist_info(store, sp)
    extract_audio_features(store, sp)
    return store

In [22]:
store = construct_storage()  # Construct info storage
store = extract_tracks(sp, playlist_uri, store)

Total songs: 301
Current offset: 0
Current offset: 100
Current offset: 200
Current offset: 300


In [23]:
for key, value in store.items(): 
    print(key, len(value))

uris 301
names 301
artist_names 301
artist_uris 301
artist_pop 301
artist_genres 301
albums 301
track_pop 301
danceability 301
energy 301
keys 301
loudness 301
modes 301
speechiness 301
acousticness 301
instrumentalness 301
liveness 301
valences 301
tempos 301
types 301
ids 301
track_hrefs 301
analysis_urls 301
durations_ms 301
time_signatures 301
playlist_name 0


In [24]:
del store['playlist_name']
df = pd.DataFrame.from_dict(store)

In [25]:
df.head()

,uris,names,artist_names,artist_uris,artist_pop,artist_genres,albums,track_pop,danceability,energy,keys,loudness,modes,speechiness,acousticness,instrumentalness,liveness,valences,tempos,types,ids,track_hrefs,analysis_urls,durations_ms,time_signatures
0,3V9cM3nCH2G66afoDi0snu,BOOM,X Ambassadors,3NPpFNZtSTHheNBaWC82rB,70,"[modern alternative rock, modern rock, stomp pop]",ORION,62,0.732,0.594,11,-5.911,0,0.0693,0.008050,0.030100,0.0802,0.676,107.996,audio_features,3V9cM3nCH2G66afoDi0snu,https://api.spotify.com/v1/tracks/3V9cM3nCH2G6...,https://api.spotify.com/v1/audio-analysis/3V9c...,164587,4
1,0fYVliAYKHuPmECRs1pbRf,Renegades,X Ambassadors,3NPpFNZtSTHheNBaWC82rB,70,"[modern alternative rock, modern rock, stomp pop]",VHS,80,0.526,0.862,2,-6.003,1,0.0905,0.014400,0.059700,0.2290,0.528,90.052,audio_features,0fYVliAYKHuPmECRs1pbRf,https://api.spotify.com/v1/tracks/0fYVliAYKHuP...,https://api.spotify.com/v1/audio-analysis/0fYV...,195200,4
2,2NldmECxe8jE4TmLmu5Yao,Hey Child,Korbee,5xaoFkc3w6UTdTZ1GJOIsG,16,[],Hey Child,34,0.649,0.796,9,-3.890,1,0.0328,0.002670,0.000157,0.3620,0.653,140.035,audio_features,2NldmECxe8jE4TmLmu5Yao,https://api.spotify.com/v1/tracks/2NldmECxe8jE...,https://api.spotify.com/v1/audio-analysis/2Nld...,241267,4
3,3xgK660fsZH7ZDcOMfIdfB,Jungle,X Ambassadors,3NPpFNZtSTHheNBaWC82rB,70,"[modern alternative rock, modern rock, stomp pop]",VHS,62,0.389,0.748,2,-4.461,1,0.0805,0.000821,0.000580,0.3580,0.216,78.056,audio_features,3xgK660fsZH7ZDcOMfIdfB,https://api.spotify.com/v1/tracks/3xgK660fsZH7...,https://api.spotify.com/v1/audio-analysis/3xgK...,189707,4
4,7o2CTH4ctstm8TNelqjb51,Sweet Child O' Mine,Guns N' Roses,3qm84nBOXUEQ2vnTfUTTFC,78,"[glam metal, hard rock, rock]",Appetite For Destruction,17,0.454,0.910,6,-7.766,1,0.0448,0.086600,0.099600,0.1160,0.629,125.116,audio_features,7o2CTH4ctstm8TNelqjb51,https://api.spotify.com/v1/tracks/7o2CTH4ctstm...,https://api.spotify.com/v1/audio-analysis/7o2C...,354520,4


## Access Top-performing Playlists per Country

Access current/ top performing playlists in order to extract playlist uri's for automated track data extraction. 

In [26]:
playlists = sp.featured_playlists(country='GB', limit=20)

In [27]:
playlist_items = playlists['playlists']['items']

In [28]:
for item in playlist_items[0:2]:
    print(item['uri'])
    print(item['name'])

spotify:playlist:37i9dQZF1DWY4lFlS4Pnso
Hot Hits UK
spotify:playlist:37i9dQZF1DXcBWIGoYBM5M
Today's Top Hits 


In [29]:
def find_top_playlists(country):
    uris = []
    names = []
    playlists = sp.featured_playlists(country=country, limit=20)
    playlist_items = playlists['playlists']['items']
    for item in playlist_items:
        uris.append(item['uri'].split(':')[-1])
        names.append(item['name'])
    return uris, names

In [30]:
top_playlists, names = find_top_playlists('GB')
print(top_playlists)
print(names)

['37i9dQZF1DWY4lFlS4Pnso', '37i9dQZF1DXcBWIGoYBM5M', '37i9dQZF1DX5KpP2LN299J', '37i9dQZF1DX5uokaTN4FTR', '37i9dQZF1DX8C9xQcOrE6T', '37i9dQZF1DX8S9gwdi7dev', '37i9dQZF1DXb57FjYWz00c', '37i9dQZF1DWZd79rJ6a7lp', '37i9dQZF1DX2L0iB23Enbq', '37i9dQZF1DX6PKX5dyBKeq', '37i9dQZF1DX0XUsuxWHRQd', '37i9dQZF1DWWMOmoXKqHTD', '37i9dQZF1DWUZ5bk6qqDSy', '37i9dQZF1DX5Vy6DFOcx00', '37i9dQZF1DWW1yuCSjnKtM', '37i9dQZF1DXbTxeAdrVG2l', '37i9dQZF1DWWY64wDtewQt', '37i9dQZF1DX0Yxoavh5qJV', '37i9dQZF1DWVlYsZJXqdym', '37i9dQZF1DWSqmBTGDYngZ']
['Hot Hits UK', "Today's Top Hits ", 'This Is Taylor Swift', 'Massive Dance Hits', 'Disney Hits', 'Halloween Party', '80s Hits', 'Sleep', 'Viral Hits', 'Rap UK', 'RapCaviar', 'Songs to Sing in the Car', 'White Noise 10 Hours', 'big on the internet', '!!!', 'All Out 90s', 'phonk', 'Christmas Hits', 'Happy Pop Hits', 'Songs to Sing in the Shower']


Confirm extracted uri's work with the created track extraction function

In [31]:
store = construct_storage()
tracks = extract_tracks(sp, top_playlists[0], store)

Total songs: 70
Current offset: 0


In [32]:
print(tracks)

{'uris': ['5aIVCx5tnk0ntmdiinnYvw', '4RoKNqyZ9622tcAeNPNv5k', '3rUGC1vUpkDG9CZFHMur1t', '5mjYQaktjmjcMKcUIcqz4s', '4bz8Z7squfs2Yji2DwoujR', '7DlZH8EiVDgQXNoj3dnyZC', '3YgtkOxZsTuaZdL8McA1FQ', '2ZWmmrWUgDBcPSLihBMvhg', '1vYXt7VSjH9JIM5oRRo7vA', '59NraMJsLaMCVtwXTSia8i', '1Fav4xZn60uGM3XmiHJmvT', '5RHIevCuoIZcY1AlnJ01fN', '6WzRpISELf3YglGAh7TXcG', '58pTPJPBjw1xL9OcLcQczU', '602d2gJewoiF1FivuOMMwE', '2FDTHlrBguDzQkp7PVj16Q', '6ibDVMcMUNqZ5eXT9sD4Vy', '4nFrcGM7MY1mpoQCC7Kefj', '23RoR84KodL5HWvUTneQ1w', '1BxfuPKGuaTgP7aM0Bbdwr', '2gyxAWHebV7xPYVxqoi86f', '0rKWJnmo6Q0ovoPOLoLm0t', '0ZVjgfaC2Ptrod9v6p9KFP', '0cVyQfDyRnMJ0V3rjjdlU3', '2N8gR2yftrHbZR53U5kYnX', '2doTn2LWTKN1Z0lZJG2WQw', '2YSzYUF3jWqb9YP9VXmpjE', '2XOJCFJ6Wig25fnEjxtSsR', '0mflMxspEfB0VbI1kyLiAv', '6GrLwf1LdBCmkBOsmOcbPp', '5VxmI3IdgAxWVvUnJoLuY2', '15BDY6GKFfk02WaoyytP9g', '1ACFweuuvf6MHtptObgreR', '4ZnkygoWLzcGbQYCm3lkae', '26b3oVLrRUaaybJulow9kz', '7DnI3ktF2vcmzKuCCKseQL', '1KNYCbS2hGara6pzGCa2Ui', '4z5mxHbpuduI70kTrpSVHE', '1

## Data Collection

Specify the countries from which to extract the top 20 playlists from: 
- South Africa (**ZA**)
- Australia (**AU**)
- United Kingdom (**GB**)
- United States (**UK**)
- Canada (**CA**)
- Jamaica (**JM**)
- Malta (**MT**)
- Netherlands (**NL**)
- France (**FR**)
- Germany (**DE**)
- Ghana (**GH**)

In [33]:
def add_playlist_tracking(name, store):
    store['playlist_name'] = [name] * len(store['uris'])
    return store


def record_playlists(top_playlists, names, playlist_store, name_store):
    playlist_store.extend(top_playlists)
    name_store.extend(names)

In [34]:
countries = ['AU', 'GB', 'US', 'CA', 'JM', 'MT', 'NL', 'FR', 'DE', 'GH', 'ZA']

In [35]:
playlist_store = []
name_store = []  # Construct playlist info storage
store = construct_storage()  # Construct track info storage

for country in countries[0:1]: 
    print(f'Country: {country}') 
    top_playlists, names = find_top_playlists(country)

    for playlist, name in zip(top_playlists[0:2], names[0:2]):
        print(f'Playlist name: {name}')
        store = extract_tracks(sp, playlist, store)
        store = add_playlist_tracking(name, store)

    record_playlists(top_playlists, names, playlist_store, name_store)
    print('-----------------------------------------------------------------------------')

Country: AU
Playlist name: Hot Hits Australia
Total songs: 50
Current offset: 0
Playlist name: Today's Top Hits 
Total songs: 50
Current offset: 0
-----------------------------------------------------------------------------


In [36]:
print(playlist_store)
print(name_store)
print("Total Playlists: ", len(name_store))

['37i9dQZF1DWXXs9GFYnvLB', '37i9dQZF1DXcBWIGoYBM5M', '37i9dQZF1DX5KpP2LN299J', '37i9dQZF1DX7aUUBCKwo4Y', '37i9dQZF1DWZd79rJ6a7lp', '37i9dQZF1DWVYHdtUb7Wil', '37i9dQZF1DX3rxVfibe1L0', '37i9dQZF1DWWMOmoXKqHTD', '37i9dQZF1DX76Wlfdnj7AP', '37i9dQZF1DX8C9xQcOrE6T', '37i9dQZF1DX0sDai2F5jCQ', '37i9dQZF1DWU7oFJkRIANi', '37i9dQZF1DWUZ5bk6qqDSy', '37i9dQZF1DWZqd5JICZI0u', '37i9dQZF1DX574gardB2et', '37i9dQZF1DX4sWSpwq3LiO', '37i9dQZF1DXdSo2zQhniZC', '37i9dQZF1DXcyVNt8nnPfR', '37i9dQZF1DWTwnEm1IYyoj', '37i9dQZF1DWWY64wDtewQt']
['Hot Hits Australia', "Today's Top Hits ", 'This Is Taylor Swift', 'Country Top 50', 'Sleep', 'Chilled Hits', 'Mood Booster', 'Songs to Sing in the Car', 'Beast Mode', 'Disney Hits', 'A1', 'Every Number One', 'White Noise 10 Hours', 'Peaceful Meditation', 'Hit Repeat', 'Peaceful Piano', 'Aussie Rock Classics', 'Chilled Classics', 'Soft Pop Hits', 'phonk']
Total Playlists:  20


In [37]:
print(store)

{'uris': ['3rUGC1vUpkDG9CZFHMur1t', '31MNHKE86sEXzIglbGQ6mu', '5aIVCx5tnk0ntmdiinnYvw', '59NraMJsLaMCVtwXTSia8i', '7DlZH8EiVDgQXNoj3dnyZC', '7aqfrAY2p9BUSiupwk3svU', '4KULAymBBJcPRpk1yO4dOG', '0cVyQfDyRnMJ0V3rjjdlU3', '672pbi6q4UuyJYIuThNsq3', '1aNUSKBe6UMyMk3pEu9ws7', '0rKWJnmo6Q0ovoPOLoLm0t', '4M5SqWR8LIgE9N3QzEJlyS', '4rDvBpFgKbgJepZTaP0rD6', '6WzRpISELf3YglGAh7TXcG', '4ZnkygoWLzcGbQYCm3lkae', '2gyxAWHebV7xPYVxqoi86f', '1yeB8MUNeLo9Ek1UEpsyz6', '0Nfh4u0J4bajcyp5pzaWCx', '7dJYggqjKo71KI9sLzqCs8', '5mjYQaktjmjcMKcUIcqz4s', '6iF4RgIjDvDqyW13PezSj3', '4YQImHflXSiIMXntcwPkx8', '6wf7Yu7cxBSPrRlWeSeK0Q', '6Ea2oEzysv4UECGNxL1IEW', '7MeniYdHjzvKEn8BWzFNI1', '1vYXt7VSjH9JIM5oRRo7vA', '3BHFResGQiUvbYToUdaDQz', '3BKD1PwArikchz2Zrlp1qi', '1Lo0QY9cvc8sUB2vnIOxDT', '3vkCueOmm7xQDoJ17W1Pm3', '1BxfuPKGuaTgP7aM0Bbdwr', '23RoR84KodL5HWvUTneQ1w', '1kuGVB7EU95pJObxwvfwKS', '741UUVE2kuITl0c6zuqqbO', '6HgWWaMu31KdOpEG5l28BG', '59uQI0PADDKeE6UZDTJEe8', '0WtM2NBVQNNJLh6scP13H8', '5eTaQYBE1yrActixMAeLcZ', '5

## Save Information
Save the information to a DataFrame and to a csv option.

Verify the identical lengths of all store information ahead of constructing a DataFrame

In [38]:
for key, value in store.items(): 
    print(key, len(value))

uris 100
names 100
artist_names 100
artist_uris 100
artist_pop 150
artist_genres 150
albums 100
track_pop 100
danceability 150
energy 150
keys 150
loudness 150
modes 150
speechiness 150
acousticness 150
instrumentalness 150
liveness 150
valences 150
tempos 150
types 150
ids 150
track_hrefs 150
analysis_urls 150
durations_ms 150
time_signatures 150
playlist_name 100


## Note
There exists a size mismatch when top-playlists are concatenated together. This issue is resolved in further playlists. However in this notebook it still needs to be resolved. This is visible in the above printout